# DeepShip 数据集加环境噪声类后的标注文件处理

In [2]:
import pandas as pd
import os
import glob
from pathlib import Path
from tqdm import tqdm

In [10]:
# 配置输入文件路径
csv_path = r'X:\\数据集\\DeepShip\\data_preprocessing\\annotation\\DeepShip.csv'
train_txt_path = r'X:\\数据集\\DeepShip\\annotation_original\\training_and_testing\\train.txt'
test_txt_path = r'X:\\数据集\\DeepShip\\annotation_original\\training_and_testing\\test.txt'
output_csv_path = r'X:\\数据集\\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata.csv'

# 读取原始 CSV
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"成功读取 CSV文件，共 {len(df)} 条记录。")
else:
    raise FileNotFoundError(f"未找到文件: {csv_path}")

成功读取 CSV文件，共 610 条记录。


In [11]:
df['new_filename'] = df.apply(lambda row: f"{row['class_id']}_{row['ID']}.wav", axis=1)

print("新文件名示例:")
print(df[['ID', 'class_id', 'new_filename']].head())

新文件名示例:
   ID  class_id new_filename
0   1         0      0_1.wav
1   2         0      0_2.wav
2   3         0      0_3.wav
3   4         0      0_4.wav
4   5         0      0_5.wav


In [ ]:
def get_split_keys(file_path):
    keys = set()
    if not os.path.exists(file_path):
        print(f"警告: 文件不存在 {file_path}")
        return keys
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            parts = line.split('/')
            if len(parts) >= 3:
                class_name = parts[-3]
                folder_name = parts[-2]
                keys.add((class_name, folder_name))
    return keys

In [14]:
# 加载训练集和测试集的标识键
train_keys = get_split_keys(train_txt_path)
test_keys = get_split_keys(test_txt_path)

print(f"从 {train_txt_path} 解析出 {len(train_keys)} 个唯一文件夹 (Train)")
print(f"从 {test_txt_path} 解析出 {len(test_keys)} 个唯一文件夹 (Test)")

从 X:\\数据集\\DeepShip\\annotation_original\\training_and_testing\\train.txt 解析出 398 个唯一文件夹 (Train)
从 X:\\数据集\\DeepShip\\annotation_original\\training_and_testing\\test.txt 解析出 211 个唯一文件夹 (Test)


In [15]:
class_mapping = {
    0: 'Cargo',
    1: 'Passengership',
    2: 'Tanker',
    3: 'Tug'
}

def assign_split_label(row):
    if pd.isna(row['folder_name']): return 'unassigned'
    txt_class = class_mapping.get(row['class_id'])
    key = (txt_class, row['folder_name'])
    
    if key in train_keys: return 'train'
    elif key in test_keys: return 'test'
    else: return 'unassigned'

df['dataset_split'] = df.apply(assign_split_label, axis=1)

# 打印统计结果
split_counts = df['dataset_split'].value_counts()
print("\n数据集划分统计:")
print(split_counts)

# 检查是否有未分配的数据 (通常是ID 23, 因为缺失folder_name)
unassigned = df[df['dataset_split'] == 'unassigned']
if len(unassigned) > 0:
    print("\n注意：以下数据未被分配 (可能缺失元数据):")
    print(unassigned[['ID', 'Ship Name', 'folder_name']])


数据集划分统计:
dataset_split
train         398
test          211
unassigned      1
Name: count, dtype: int64

注意：以下数据未被分配 (可能缺失元数据):
    ID Ship Name folder_name
21  23   GALLEON         NaN


In [21]:
df_clean = df[df['dataset_split'] != 'unassigned'].copy()
print(f"已剔除未分配数据 {len(df) - len(df_clean)} 条。")

target_columns = [
    'ID', 
    'new_filename',
    'class_id', 
    'class ID', 
    'Ship Name', 
    'folder_name', 
    'Date & Time',
    'Duration(sec)',
    'Distances(m)',
    'prompt_en',
    'dataset_split'
]

# 确保列都存在
df_final = df_clean[target_columns]

# 打印统计信息
print("\n最终划分统计结果:")
print(df_final['dataset_split'].value_counts())

# 保存文件
df_final.to_csv(output_csv_path, index=False)
print(f"\n处理完成！文件已保存为: {output_csv_path}")

已剔除未分配数据 1 条。

最终划分统计结果:
dataset_split
train    398
test     211
Name: count, dtype: int64

处理完成！文件已保存为: X:\\数据集\\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata.csv


# 处理海洋环境噪声的标注信息

In [26]:
original_csv_path = r"X:\\数据集\\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata.csv"
output_csv_path = r"X:\数据集\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata_add_Environmental_Noise.csv"

# 噪声数据的根目录
NOISE_ROOT_PATH = r'X:\数据集\DeepShip\data_original\background'

In [27]:
df_ship = pd.read_csv(original_csv_path)
print(f"原始 4 类数据加载完成: {len(df_ship)} 条记录")

原始 4 类数据加载完成: 609 条记录


In [ ]:
print("=== 开始执行文件重命名 ===")
def rename_noise_files(root_path):
    count = 0
    errors = 0
    
    # 遍历 train 和 test
    for split in ['train', 'test']:
        split_dir = os.path.join(root_path, split)
        if not os.path.exists(split_dir):
            print(f"警告: 找不到目录 {split_dir}")
            continue
            
        print(f"正在扫描目录: {split_dir}")
        
        # 使用 os.walk 遍历
        for root, dirs, files in os.walk(split_dir):
            for file in files:
                if file.endswith('.wav'):
                    # 检查文件名是否已经是目标格式 (防止重复运行导致错误)
                    if file.startswith('4_') and file.count('_') >= 2:
                        continue
                        
                    old_path = os.path.join(root, file)
                    
                    try:
                        # === 解析路径逻辑 ===
                        # 示例路径: ...\3_0\0000\00000.wav
                        path_parts = os.path.normpath(old_path).split(os.sep)
                        
                        # 1. 提取 FileID (00000 -> 1)
                        file_stem = os.path.splitext(file)[0]
                        try:
                            file_num_id = int(file_stem) + 1 # 00000 -> 1
                        except ValueError:
                            # 如果文件名不是纯数字，可能需要特殊处理，或者跳过
                            print(f"跳过非数字文件名: {file}")
                            continue

                        # 2. 提取 FolderID (3_0 -> 0)
                        # 寻找包含 '_' 的上级目录
                        parent = path_parts[-2]      # 0000
                        grandparent = path_parts[-3] # 3_0
                        
                        target_folder = grandparent if '_' in grandparent else parent
                        
                        if '_' in target_folder:
                            folder_id_part = target_folder.split('_')[-1]
                        else:
                            # 如果找不到下划线，尝试用当前目录名
                            folder_id_part = target_folder
                            
                        # === 构造新文件名 ===
                        # 格式: 4_{FolderID}_{FileID}.wav
                        new_filename = f"4_{folder_id_part}_{file_num_id}.wav"
                        new_path = os.path.join(root, new_filename)
                        
                        # === 执行重命名 ===
                        if old_path != new_path:
                            os.rename(old_path, new_path)
                            count += 1
                            if count % 100 == 0:
                                print(f"已重命名 {count} 个文件...", end='\r')
                                
                    except Exception as e:
                        print(f"重命名失败: {old_path}, 错误: {e}")
                        errors += 1

    print(f"\n重命名完成! 共修改 {count} 个文件, 失败 {errors} 个。")
    
# 执行重命名函数
rename_noise_files(NOISE_ROOT_PATH)

=== 开始执行文件重命名 ===
正在扫描目录: X:\数据集\DeepShip\data_original\background\train
正在扫描目录: X:\数据集\DeepShip\data_original\background\test
已重命名 27700 个文件...
重命名完成! 共修改 27768 个文件, 失败 0 个。


In [37]:
print("\n=== 开始生成标注信息 ===")

if os.path.exists(original_csv_path):
    df_ship = pd.read_csv(original_csv_path)
    print("成功读取原始数据。")
else:
    raise FileNotFoundError(f"未找到原始文件: {original_csv_path}")

# 扫描已重命名的文件
def scan_renamed_files(root_path):
    noise_rows = []
    # === 修改处：ID 从 1 开始独立计数 ===
    current_id = 1 
    
    for split in ['train', 'test']:
        split_dir = os.path.join(root_path, split)
        if not os.path.exists(split_dir): continue
        
        for root, dirs, files in os.walk(split_dir):
            for file in files:
                if file.endswith('.wav') and file.startswith('4_'):
                    
                    # 获取 folder_name
                    path_parts = os.path.normpath(root).split(os.sep)
                    parent = path_parts[-1] 
                    grandparent = path_parts[-2] 
                    raw_folder_name = grandparent if '_' in grandparent else parent

                    row = {
                        'ID': current_id,
                        'new_filename': file,  
                        'class_id': 4,
                        'class ID': None,
                        'Ship Name': 'Marine Environmental Noise',
                        'folder_name': raw_folder_name, 
                        'Date & Time': 'Unknown',
                        'Duration(sec)': 3,
                        'Distances(m)': None,
                        'prompt_en': 'Hydrophone recording of marine environmental noise.',
                        'dataset_split': split
                    }
                    noise_rows.append(row)
                    # 计数器递增
                    current_id += 1
                    
    return pd.DataFrame(noise_rows)

# 执行扫描
df_noise = scan_renamed_files(NOISE_ROOT_PATH)


=== 开始生成标注信息 ===
成功读取原始数据。


In [38]:
if len(df_noise) > 0:
    print(f"扫描到 {len(df_noise)} 条噪声数据。")
    print(f"噪声数据的 ID 范围: {df_noise['ID'].min()} - {df_noise['ID'].max()}")
    
    # 2.3 合并数据
    target_columns = [
        'ID', 'new_filename', 'class_id', 'class ID', 
        'Ship Name', 'folder_name', 'Date & Time', 
        'Duration(sec)', 'Distances(m)', 'prompt_en', 'dataset_split'
    ]
    
    # 确保列一致
    df_ship_aligned = df_ship[target_columns]
    df_noise_aligned = df_noise[target_columns]
    
    # 合并
    df_final = pd.concat([df_ship_aligned, df_noise_aligned], ignore_index=True)
    
    # 保存
    df_final.to_csv(output_csv_path, index=False)
    print(f"\n最终处理完成！文件已保存至: {output_csv_path}")
else:
    print("未找到任何噪声文件。")

扫描到 27768 条噪声数据。
噪声数据的 ID 范围: 1 - 27768

最终处理完成！文件已保存至: X:\数据集\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata_add_Environmental_Noise.csv


# 根据切割好的音频重新处理标注信息

In [7]:
ORIGINAL_CSV_PATH = r"X:\数据集\DeepShip\data_preprocessing\annotation\DeepShip_No_Overlap_Metadata_add_Environmental_Noise.csv"

PREPROCESSED_ROOT = Path(r"X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_dataset_split_preprocessed")

OUTPUT_CSV_PATH = r"X:\数据集\DeepShip\data_preprocessing\annotation\DeepShip_No_Overlap_add_Environmental_Noise_Segmented_Metadata.csv"

In [5]:
def load_metadata_lookup(csv_path):
    """
    加载原始 CSV 并建立查找表
    Key: 文件名 stem (不含后缀), 例如 "0_3" 或 "4_0_1"
    Value: 该行的 Series 数据
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"找不到原始标注文件: {csv_path}")
    
    df = pd.read_csv(csv_path)
    lookup = {}
    
    # 建立 filename_stem -> row 的映射
    for idx, row in df.iterrows():
        # 获取 new_filename 的 stem (例如 "0_3.wav" -> "0_3")
        fname = str(row['new_filename'])
        if fname.lower().endswith('.wav'):
            stem = fname[:-4]
        else:
            stem = fname
        lookup[stem] = row
    
    print(f"原始元数据加载完成，共 {len(lookup)} 条索引。")
    return lookup

def generate_segmented_metadata():
    # 1. 加载查找表
    try:
        meta_lookup = load_metadata_lookup(ORIGINAL_CSV_PATH)
    except Exception as e:
        print(f"❌ 错误: {e}")
        return

    # 2. 准备新数据列表
    new_rows = []
    
    # 3. 遍历预处理后的文件夹
    # 期望结构: root / split / category / file.wav
    if not PREPROCESSED_ROOT.exists():
        print(f"❌ 错误: 预处理目录不存在 {PREPROCESSED_ROOT}")
        return

    print("🚀 开始扫描并生成新标注...")
    
    # 全局 ID 计数器 (新文件从 1 开始编号)
    global_id_counter = 1
    
    # 遍历 train 和 test
    for split in ['train', 'test']:
        split_dir = PREPROCESSED_ROOT / split
        if not split_dir.exists(): continue
        
        # 遍历类别 (Cargo, Background, ...)
        for category_dir in split_dir.iterdir():
            if not category_dir.is_dir(): continue
            
            category_name = category_dir.name
            
            # 判断是否为噪声类 (Background)
            # 如果是噪声类，文件名没有变 (4_0_1.wav -> 4_0_1)
            # 如果是船只类，文件名变了 (0_3.wav -> 0_3_1)
            is_noise = (category_name == "Background")
            
            files = sorted(list(category_dir.glob("*.wav")))
            
            for wav_file in tqdm(files, desc=f"Scanning {split}/{category_name}", leave=False):
                current_stem = wav_file.stem  # 例如 "0_3_1" 或 "4_0_1"
                
                # === A. 确定父文件的 Key ===
                parent_key = None
                
                if is_noise:
                    # 噪声类文件名未变，直接查
                    parent_key = current_stem
                else:
                    # 船只类文件名多了后缀 "_1", "_2"
                    # 解析逻辑: 0_3_1 -> 0_3
                    # 使用 rsplit 切分最后一个下划线
                    if '_' in current_stem:
                        parent_key = current_stem.rsplit('_', 1)[0]
                    else:
                        # 异常情况，防报错
                        parent_key = current_stem
                
                # === B. 查找元数据 ===
                parent_info = meta_lookup.get(parent_key)
                
                if parent_info is None:
                    # 如果查不到，打印警告 (可能有些文件被删了或者命名对不上)
                    # print(f"⚠️ 警告: 找不到父样本信息: {parent_key} (当前文件: {wav_file.name})")
                    continue
                
                # === C. 构建新行 ===
                new_row = parent_info.copy()
                
                # 更新关键字段
                new_row['ID'] = global_id_counter
                new_row['new_filename'] = wav_file.name # 更新为切片后的文件名
                new_row['Duration(sec)'] = 3.0          # 统一修正为 3秒
                new_row['dataset_split'] = split        # 以实际所在文件夹为准
                
                # 可以选择保留 parent_filename 以便追溯
                new_row['parent_filename'] = parent_info['new_filename']
                
                new_rows.append(new_row)
                global_id_counter += 1

    # 4. 生成 DataFrame 并保存
    if not new_rows:
        print("❌ 未生成任何数据，请检查路径或文件名匹配规则。")
        return

    df_final = pd.DataFrame(new_rows)
    
    # 调整列顺序 (把 parent_filename 放在后面，ID 放在最前)
    cols = list(df_final.columns)
    if 'parent_filename' in cols:
        cols.remove('parent_filename')
        cols.append('parent_filename') # 移到最后
    
    # 确保 ID 在第一列
    if 'ID' in cols:
        cols.remove('ID')
        cols.insert(0, 'ID')
        
    df_final = df_final[cols]

    # 保存
    df_final.to_csv(OUTPUT_CSV_PATH, index=False)
    
    print("\n" + "="*30)
    print("✅ 新标注文件生成成功！")
    print(f"📄 输出文件: {OUTPUT_CSV_PATH}")
    print(f"📊 总样本数: {len(df_final)}")

In [8]:
# 运行
generate_segmented_metadata()

原始元数据加载完成，共 23001 条索引。
🚀 开始扫描并生成新标注...



✅ 新标注文件生成成功！
📄 输出文件: X:\数据集\DeepShip\data_preprocessing\annotation\DeepShip_No_Overlap_add_Environmental_Noise_Segmented_Metadata.csv
📊 总样本数: 84236


In [11]:
# 新生成的标注文件
CSV_PATH = r"X:\数据集\DeepShip\data_preprocessing\annotation\DeepShip_No_Overlap_add_Environmental_Noise_Segmented_Metadata.csv"

# 预处理后的音频根目录
DATA_ROOT = Path(r"X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_dataset_split_preprocessed")

# 类别映射
ID_TO_FOLDER = {
    0: 'Cargo',
    1: 'Passengership',
    2: 'Tanker',
    3: 'Tug',
    4: 'Background'
}

# ================= 校验逻辑 =================
def verify_dataset_integrity():
    print("🚀 开始执行数据集完整性校验...\n")
    
    # --- 步骤 1: 读取 CSV ---
    if not os.path.exists(CSV_PATH):
        print(f"❌ 致命错误: 找不到标注文件 {CSV_PATH}")
        return
    
    df = pd.read_csv(CSV_PATH)
    print(f"📄 CSV 记录总数: {len(df)}")
    
    # 构建 CSV 中的文件集合: set of (split, category_name, filename)
    csv_index = set()
    csv_counts = {} # 用于统计各类别的数量
    
    for _, row in df.iterrows():
        split = row['dataset_split']
        class_id = row['class_id']
        fname = row['new_filename']
        
        cat_name = ID_TO_FOLDER.get(class_id)
        if not cat_name:
            print(f"❌ CSV 包含未知的 class_id: {class_id}")
            continue
            
        # 记录关键特征元组
        key = (split, cat_name, fname)
        csv_index.add(key)
        
        # 统计
        if cat_name not in csv_counts: csv_counts[cat_name] = 0
        csv_counts[cat_name] += 1

    # --- 步骤 2: 扫描硬盘 ---
    if not DATA_ROOT.exists():
        print(f"❌ 致命错误: 数据目录不存在 {DATA_ROOT}")
        return

    print(f"📂 正在扫描硬盘目录: {DATA_ROOT} ...")
    disk_index = set()
    disk_counts = {}
    
    # 遍历 train 和 test
    for split in ['train', 'test']:
        split_dir = DATA_ROOT / split
        if not split_dir.exists(): continue
        
        # 遍历类别
        for cat_dir in split_dir.iterdir():
            if not cat_dir.is_dir(): continue
            cat_name = cat_dir.name
            
            # 扫描文件
            files = list(cat_dir.glob("*.wav"))
            
            # 统计
            if cat_name not in disk_counts: disk_counts[cat_name] = 0
            disk_counts[cat_name] += len(files)
            
            for f in files:
                # 记录关键特征元组
                key = (split, cat_name, f.name)
                disk_index.add(key)

    print(f"💾 硬盘文件总数: {len(disk_index)}")

    # --- 步骤 3: 核心对比 ---
    print("\n" + "="*40)
    print("📊 校验结果报告")
    print("="*40)

    # 检查 CSV 中有，但硬盘没有的 (Missing Files)
    missing_on_disk = csv_index - disk_index
    if len(missing_on_disk) == 0:
        print("✅ [一致] CSV 中记录的所有文件都在硬盘上找到了。")
    else:
        print(f"❌ [不一致] 硬盘缺失文件数: {len(missing_on_disk)}")
        print("   前5个缺失示例:")
        for item in list(missing_on_disk)[:5]:
            print(f"   - 预期位置: {item[0]}/{item[1]}/{item[2]}")

    # 检查硬盘上有，但 CSV 没记录的 (Orphan Files)
    orphan_files = disk_index - csv_index
    if len(orphan_files) == 0:
        print("✅ [一致] 硬盘上的所有文件都在 CSV 中有记录。")
    else:
        print(f"⚠️ [警告] 发现未标注的多余文件数: {len(orphan_files)}")
        print("   这些文件存在于文件夹中，但不在 CSV 里（可能需要删除或重新生成CSV）。")
        print("   前5个多余示例:")
        for item in list(orphan_files)[:5]:
            print(f"   - 实际位置: {item[0]}/{item[1]}/{item[2]}")

    # 数量统计对比
    print("\n📈 类别数量对比 (CSV vs Disk):")
    all_cats = set(csv_counts.keys()) | set(disk_counts.keys())
    for cat in sorted(list(all_cats)):
        c_count = csv_counts.get(cat, 0)
        d_count = disk_counts.get(cat, 0)
        status = "✅ OK" if c_count == d_count else "❌ Mismatch"
        print(f"   - {cat:<15}: CSV={c_count:<6} | Disk={d_count:<6} | {status}")

    # --- 最终结论 ---
    print("\n" + "-"*40)
    if len(missing_on_disk) == 0 and len(orphan_files) == 0:
        print("🎉 验证通过！数据集与标注文件完美匹配。")
        print("   您可以放心开始训练了。")
    else:
        print("🚫 验证失败！请检查上述错误信息。")
        print("   建议：重新运行生成标注文件的脚本，或者检查预处理过程中是否有文件写入失败。")

In [12]:
# 运行
verify_dataset_integrity()

🚀 开始执行数据集完整性校验...

📄 CSV 记录总数: 84236
📂 正在扫描硬盘目录: X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_dataset_split_preprocessed ...
💾 硬盘文件总数: 84236

📊 校验结果报告
✅ [一致] CSV 中记录的所有文件都在硬盘上找到了。
✅ [一致] 硬盘上的所有文件都在 CSV 中有记录。

📈 类别数量对比 (CSV vs Disk):
   - Background     : CSV=27768  | Disk=27768  | ✅ OK
   - Cargo          : CSV=12801  | Disk=12801  | ✅ OK
   - Passengership  : CSV=15410  | Disk=15410  | ✅ OK
   - Tanker         : CSV=14762  | Disk=14762  | ✅ OK
   - Tug            : CSV=13495  | Disk=13495  | ✅ OK

----------------------------------------
🎉 验证通过！数据集与标注文件完美匹配。
   您可以放心开始训练了。
